In [22]:
# from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Sampler, Options
from qiskit.circuit import ParameterVector, QuantumCircuit
from qiskit_aer import Aer
from qiskit_algorithms import MinimumEigensolver, NumPyMinimumEigensolver, VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import Tsp
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_aer import AerSimulator
from qiskit.primitives import StatevectorEstimator
import numpy as np

In [13]:
# Nは拠点数
def tsp_entangler(N):
    
    # 量子回路とパラメータθの設定
    circuit = QuantumCircuit(N**2)
    theta =  ParameterVector('theta',length=(N-1)*N)

    for k in range(N):
        circuit.x(k*N) #手順1
    for k in range(N):
        a = k*N
        b = k*(N-1)
        for i in range(N-1): #手順5
            circuit.ry(theta[i+b],i+1+a) #手順2
            circuit.cz(i+a,i+1+a) #手順3
            circuit.ry(-theta[i+b],i+1+a) #手順4
        for i in range(N-1):
            circuit.cx(i+1+a, i+a) #手順6

    return circuit


In [14]:
n = 4

PQC = tsp_entangler(n)

In [15]:
def RunVQE(estimator, model, optimizer, operator, init=None):
    
    # 一時的な解を保存
    history = {"eval_count": [], "parameters": [], "mean": [], "metadata": []}
    def store_intermediate_result(eval_count, parameters, mean, metadata):
        history["eval_count"].append(eval_count)
        history["parameters"].append(parameters)
        history["mean"].append(mean.real)
            
    # VQE実行オブジェクトの作成
    vqe = VQE(estimator=estimator, ansatz=model, optimizer=optimizer, initial_point=init)

    # VQEによる最小固有値の求解
    result = vqe.compute_minimum_eigenvalue(operator=operator)    

    return result, history["mean"]


In [ ]:
backend = AerSimulator()

# パラメータ収束のための古典的アルゴリズムの指定(Classical Optimizer)
optimizer = SPSA(maxiter=50)

# 初期パラメータの指定
np.random.seed(10)
init = np.random.rand((n-1)*n) * 2 * np.pi

# --- ローカル実行用: qiskit.primitives の StatevectorEstimator を使う (ローカルテスト向け) ---
estimator = StatevectorEstimator()

# --- IBM runtime に切り替える場合（後で戻す時の参考） ---
# コメント: IBM Runtime を使うには Session を作成して Estimator に session=session を渡します。
# 例:
# service = QiskitRuntimeService()
# with Session(service=service, backend='ibmq_qasm_simulator') as session:
#     estimator = Estimator(session=session)
#     vqe = VQE(estimator=estimator, ansatz=PQC, optimizer=optimizer, initial_point=init)
#     result = vqe.compute_minimum_eigenvalue(operator=operator)
#     print(result)
# 上のブロックを使う場合は、下のローカル用のコードはコメントアウトしてください。

# ゲート(演算子)の指定(=TSPから変換したハミルトニアン)
operator = qubitOp

# VQEの実行（ローカル）
vqe = VQE(estimator=estimator, ansatz=PQC, optimizer=optimizer, initial_point=init)
result = vqe.compute_minimum_eigenvalue(operator=operator)

# VQEの実行結果を表示
print(result)


NameError: name 'qubitOp' is not defined